# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", AzureChatCompletion("gpt-35-turbo", endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt4", OpenAIChatCompletion("gpt-4", api_key, org_id))

## It all begins with an ask

In [2]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [3]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [4]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [5]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [7]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [8]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [9]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [10]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [11]:
results = await planner.execute_plan_async(new_plan, kernel)

In [12]:
print(results)

ASSUREZ-VOUS D'UTILISER UNIQUEMENT LE FRANÇAIS.

BONJOUR,

JE VOULAIS PARTAGER QUELQUES IDÉES D'ACTIVITÉS ROMANTIQUES QUE VOUS ET VOTRE PARTENAIRE POURRIEZ APPRÉCIER.

1. VOUS POURRIEZ AVOIR UN DÎNER CHARMANT DANS UN RESTAURANT CHIC, AVEC DES BOUGIES ET DE DÉLICIEUX PLATS.
2. ALTERNATIVEMENT, VOUS POURRIEZ PRÉPARER UN PIQUE-NIQUE ET VOUS RENDRE DANS UN BEAU PARC POUR UNE JOURNÉE DE DÉTENTE ET D'AIR FRAIS.
3. SI VOUS PRÉFÉREZ UNE SOIRÉE CONFORTABLE À LA MAISON, VOUS POURRIEZ REGARDER UN FILM ENSEMBLE ET PRÉPARER DES COLLATIONS MAISON.
4. POUR UNE EXPÉRIENCE PLUS INDULGENTE, VOUS POURRIEZ RÉSERVER UNE JOURNÉE DANS UN SPA ET PROFITER DE SOINS ENSEMBLE.
5. SI VOUS ÊTES D'HUMEUR AVENTUREUSE, VOUS POURRIEZ ESSAYER DE PRENDRE UN COURS DE DANSE ENSEMBLE.
6. UNE AUTRE OPTION EXCITANTE POURRAIT ÊTRE UN VOL EN MONTGOLFIÈRE, OÙ VOUS POURREZ ADMIRER DES VUES À COUPER LE SOUFFLE D'EN HAUT.
7. SI VOUS AIMEZ TOUS LES DEUX LA NATURE, VOUS POURRIEZ FAIRE UNE RANDONNÉE PITTORESQUE ENSEMBLE.
8. POUR UNE S

# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [18]:
from semantic_kernel.planning import SequentialPlanner
planner = SequentialPlanner(kernel)

In [14]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [15]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'_variables': {'input': ''}, '_main_key': 'input'}
Translate the input into a language of your choice : {'_variables': {'input': ''}, '_main_key': 'input'}
Turn a scenario into a short and entertaining poem. : {'_variables': {'input': ''}, '_main_key': 'input'}
Convert a string to uppercase. : {'_variables': {'input': ''}, '_main_key': 'input'}
Turn bullet points into an email to someone, using a polite tone : {'_variables': {'input': ''}, '_main_key': 'input'}


Let's ask the sequential planner to execute the plan.

In [16]:
result = await sequential_plan.invoke_async()

In [17]:
print(result)

Hello,

I wanted to share some date ideas for Valentine's Day that you might enjoy. These include a romantic dinner at a fancy restaurant, a picnic in the park, a movie night at home, a couples massage at a spa, a sunset walk on the beach, wine tasting at a vineyard, a cooking class together, ice skating or roller skating, a hot air balloon ride, and a visit to a museum or art gallery.

To add a little extra flair, I've also included some Shakespearean poems to go with each idea. For example, if you choose to have a fancy dinner, you might appreciate this poem: "A fancy dinner, oh so divine, but my date's garlic breath, not so fine. I tried to ignore, but it made my eyes water, next time, I'll suggest Italian, with wine."

Additionally, I've translated the date ideas into French and put them in uppercase for your convenience.

I hope you find these ideas helpful and enjoyable!

Best regards,
[Sender]


# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [19]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [20]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x23be5301d20>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x23be5301e10>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x23be53024d0>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x23be5370280>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x23be53704c0>}

In [21]:
 ask = "What is the sum of 110 and 990?"

In [22]:
plan = await planner.create_plan_async(goal=ask)

In [23]:
 result = await plan.invoke_async()

In [24]:
print(result)

1100
